## **Importação bibliotecas**

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 200)
import re
from datetime import datetime
import pytz

## **Funções**

In [ ]:
def filtrar_episodios_podcast(df: pd.DataFrame, coluna: str) -> pd.DataFrame:    
    # Padrões desejados
    padrao_podcast = (
        r"(PODPAH\s*(-\s*)?#\s*\d+)|"
        r"(FLOW\s*(-\s*)?#\s*\d+)|"
        r"(PODCAST\s*(-\s*)?#\s*\d+)|"
        r"(LTDA\.?\s*(-\s*)?#\s*\d+)|"
        r"(VERÃO\s*(-\s*)?#\s*\d+)"
    )
    padrao_excecao = r"RODRIGO CONSTANTINO"
    
    # Padrões indesejados
    padrao_indesejado = r"(EXTRA\s*FLOW)"
    
    # Selecionar episódios desejados
    df = df[df[coluna].apply(
        lambda titulo: (
            not pd.isna(titulo) and (
                bool(re.search(padrao_podcast, str(titulo).strip().upper())) or
                bool(re.search(padrao_excecao, str(titulo).strip().upper()))
            )
        )
    )].copy()
    
    # Remover episódios indesejados
    df = df[df[coluna].apply(
        lambda titulo: (
            pd.isna(titulo) or
            not bool(re.search(padrao_indesejado, str(titulo).strip().upper()))
        )
    )].copy()
    
    return df

def atualizar_canal_por_titulo(df: pd.DataFrame, col_titulo: str, col_canal: str) -> pd.DataFrame:
    # Padrões desejados
    padroes_flow_1 = (
        r"FLOW\s+PODCAST\s*#\s*\d+",
        r"RODRIGO CONSTANTINO"
    )
    padroes_flow_2 = (
        r"FLOW\s*#\s*\d+",
    )
    
    def definir_canal(row):
        titulo = str(row[col_titulo]).strip().upper()
        
        if any(re.search(p, titulo) for p in padroes_flow_1):
            return "Flow 1.0"
        elif any(re.search(p, titulo) for p in padroes_flow_2):
            return "Flow 2.0"
        else:
            return row[col_canal]
    
    df[col_canal] = df.apply(definir_canal, axis=1)
    
    return df

def limpar_titulo_podcast(df: pd.DataFrame, coluna: str) -> pd.DataFrame:
    # Regra 1: Transformar tudo em maiúsculo
    df[coluna] = df[coluna].astype(str).str.upper()
    
    # Regra 2: Remover tudo a partir do emoji 🤝
    df[coluna] = df[coluna].str.split("🤝").str[0]
    
    # Regra 3: Aplicar strip()
    df[coluna] = df[coluna].str.strip()
    
    return df


def extrair_nome_convidado(df: pd.DataFrame, coluna_titulo: str) -> pd.DataFrame:
    padrao_remover = (
        r"\s*[-–]\s*PODPAH\s*(-\s*)?#\s*\d+\s*$|"
        r"\s*[-–]\s*INTELIGÊNCIA\s*LTDA\.?\s*(PODCAST)?\s*#\s*\d+\s*$|"
        r"\s*[-–]\s*FLOW\s*(-\s*)?#\s*\d+\s*$|"
        r"\s*PODPAH\s*(-\s*)?#\s*\d+\s*$|"
        r"\s*FLOW\s*(-\s*)?#\s*\d+\s*$|"
        r"\s*INTELIGÊNCIA\s*LTDA\.?\s*#\s*\d+\s*$|"
        r"\s*DE VERÃO\s*#\s*\d+\s*$"
    )
    
    df["PODCAST_CONVIDADO"] = (
        df[coluna_titulo]
        .astype(str)
        .str.strip()
        .str.replace(padrao_remover, "", flags=re.IGNORECASE, regex=True)
        .str.strip()
    )
    
    return df

def extrair_numero_episodio(titulo):
    if pd.isna(titulo):
        return None

    titulo = str(titulo).strip()

    # Exceção explícita
    if titulo.upper().startswith("RODRIGO CONSTANTINO"):
        return 486

    # Padrões robustos para todos os formatos conhecidos
    padroes = [
        r'#\s*(\d{1,4})',  # "#123" ou "# 123"
        r'Flow\s+(Podcast\s+)?#\s*(\d{1,4})',
        r'Podpah\s*[-–]?\s*#\s*(\d{1,4})',
        r'Podpah\s+de\s+Verão\s+#\s*(\d{1,4})',
        r'Inteligência\s+Ltda\.?\s+(Podcast\s+)?#\s*(\d{1,4})'
    ]

    for padrao in padroes:
        match = re.search(padrao, titulo, flags=re.IGNORECASE)
        if match:
            # Usa o último grupo numérico capturado
            numeros = [g for g in match.groups() if g and g.isdigit()]
            if numeros:
                return int(numeros[-1])

    return None


def limpar_valor_numerico(valor):
    try:
        return int(str(valor).replace("'", "").replace(".", "").strip())
    except:
        return 0

def converter_para_horario_brasil(data_iso):
    if pd.isna(data_iso):
        return None

    try:
        data_utc = pd.to_datetime(data_iso, utc=True)
#        data_utc = pd.to_datetime.strptime(data_iso, "%Y-%m-%dT%H:%M:%SZ")
        fuso_brasil = pytz.timezone("America/Sao_Paulo")
        data_brasil = data_utc.replace(tzinfo=pytz.utc).astimezone(fuso_brasil)
        return data_brasil
    except Exception as e:
        print(f"Erro ao converter data: {data_iso} -> {e}")
        return None

def extrair_horario_publicacao(data_brasil):
    if data_brasil:
        return data_brasil.strftime("%H:%M")
    return None

def obter_dia_da_semana(data_brasil):
    if data_brasil:
        dias = ['segunda-feira', 'terça-feira', 'quarta-feira', 'quinta-feira', 'sexta-feira', 'sábado', 'domingo']
        return dias[data_brasil.weekday()]
    return None

def classificar_momento_do_dia(data_brasil):
    if data_brasil:
        hora = data_brasil.hour
        if 6 <= hora < 12:
            return "DIA"
        elif 12 <= hora < 18:
            return "TARDE"
        elif 18 <= hora < 24:
            return "NOITE"
        else:
            return "MADRUGADA"
    return None

def calcular_dias_desde_publicacao(data_brasil):
    if data_brasil:
        hoje = datetime.now().replace(tzinfo=None)
        return (hoje - data_brasil).days
    return None

def extrair_numero_episodio(df: pd.DataFrame, col_titulo: str) -> pd.DataFrame:
    # Regex para capturar o número após o #
    padrao_numero = r"#\s*(\d+)"
    
    # Extrair número após hashtag
    df["PODCAST_EPISODIO"] = (
        df[col_titulo]
        .astype(str)
        .str.upper()
        .apply(lambda x: re.search(padrao_numero, x).group(1) if re.search(padrao_numero, x) else None)
    )   
    
    # Tratar exceção do RODRIGO CONSTANTINO
    df.loc[
        df[col_titulo].astype(str).str.upper().str.strip() == "RODRIGO CONSTANTINO",
        "PODCAST_EPISODIO"
    ] = "486"
    
    return df

def identificar_episodios_faltantes(df: pd.DataFrame, col_canal: str, col_ep: str) -> pd.DataFrame:
    # Converte para número, ignora episódios que não são numéricos
    df = df.copy()
    df[col_ep] = pd.to_numeric(df[col_ep], errors="coerce")
    
    # Resultado final
    faltantes = []

    # Agrupa por canal
    for canal, grupo in df.groupby(col_canal):
        episodios_presentes = grupo[col_ep].dropna().astype(int).sort_values()
        
        if episodios_presentes.empty:
            continue
        
        min_ep = episodios_presentes.min()
        max_ep = episodios_presentes.max()
        
        todos = set(range(min_ep, max_ep + 1))
        existentes = set(episodios_presentes.tolist())
        ausentes = sorted(todos - existentes)
        
        for ep in ausentes:
            faltantes.append({
                col_canal: canal,
                "EPISODIO_FALTANTE": ep
            })
    
    return pd.DataFrame(faltantes)


## **Leitura**

In [9]:
df_podcasts = pd.read_excel("../../dados/tabelas/01_EPS_COMPLETOS_RAW.xlsx")
df_podcasts.head(2)

,CANAL,VIDEO_TITULO,ID_VIDEO,DURACAO,DATA_PUBLICACAO,VISUALIZACOES,CURTIDAS,COMENTARIOS,CATEGORIA_ID,DESCRICAO_CATEGORIA,DT_EXECUCAO_SCRIPT,STATUS_EXTRAÇÃO
0,Flow Podcast,REINALDO AZEVEDO - Flow #455,a5M5Je7yKqE,11272.0,2025-06-07T04:00:25Z,234101,20631,1560,22,People & Blogs,2025-06-10 11:10:33,Sucesso
1,Flow Podcast,PVC - Flow #453,8alvXuLKQzA,9036.0,2025-06-04T00:12:44Z,191972,10072,359,22,People & Blogs,2025-06-10 11:10:33,Sucesso


## **Transformação**

In [ ]:
df = df_podcasts.copy()
df = filtrar_episodios_podcast(df, 'VIDEO_TITULO')  
df = atualizar_canal_por_titulo(df, 'VIDEO_TITULO', 'CANAL')
df = limpar_titulo_podcast(df, 'VIDEO_TITULO')
df = extrair_nome_convidado(df, 'VIDEO_TITULO')
df = extrair_numero_episodio(df, 'VIDEO_TITULO')
df = df.drop("STATUS_EXTRAÇÃO", axis=1)

# Limpar valores numéricos
df['VISUALIZACOES'] = df['VISUALIZACOES'].apply(limpar_valor_numerico)
df['CURTIDAS'] = df['CURTIDAS'].apply(limpar_valor_numerico)
df['COMENTARIOS'] = df['COMENTARIOS'].apply(limpar_valor_numerico)

# Calcular métricas
df['CURTIDAS_POR_VISUALIZACOES'] = df.apply(lambda row: row['CURTIDAS'] / row['VISUALIZACOES'] if row['VISUALIZACOES'] > 0 else 0, axis=1)
df['COMENTARIOS_POR_VISUALIZACOES'] = df.apply(lambda row: row['COMENTARIOS'] / row['VISUALIZACOES'] if row['VISUALIZACOES'] > 0 else 0, axis=1)

# Converter para datetime no fuso horário do Brasil
df['DATA_PUBLICACAO_BR'] = df['DATA_PUBLICACAO'].apply(converter_para_horario_brasil)
df['DATA_PUBLICACAO_BR'] = df['DATA_PUBLICACAO_BR'].dt.tz_localize(None)
df['MES_ANO_PUBLICACAO'] = df['DATA_PUBLICACAO_BR'].dt.strftime('%m/%Y')
df['ANO_PUBLICACAO'] = df['DATA_PUBLICACAO_BR'].dt.strftime('%Y')
df['MES_PUBLICACAO'] = df['DATA_PUBLICACAO_BR'].dt.strftime('%m')
df['HORARIO_PUBLICACAO'] = df['DATA_PUBLICACAO_BR'].apply(extrair_horario_publicacao)
df['DIA_DA_SEMANA'] = df['DATA_PUBLICACAO_BR'].apply(obter_dia_da_semana)
df['MOMENTO_DIA'] = df['DATA_PUBLICACAO_BR'].apply(classificar_momento_do_dia)
df['DIAS_DESDE_PUBLICACAO'] = df['DATA_PUBLICACAO_BR'].apply(calcular_dias_desde_publicacao)

# Ideias
# QTD_CORTES -> Trazer quantidade de cortes daquele vídeo
# QTD_VISUALIZACOES_CORTES ->
# QTD_CURTIDAS_CORTES -> 
# QTD_COMENTARIOS_CORTES ->
# DESEMPENHO_VIDEO -> 0 ~ 100K = BAIXO, 100K~500K = MEDIO_BAIXO, 500K~1M = MEDIO_ALTO, 1M+ = ALTO ou classificar por quartis
# CATEGORIA_DURACAO_VIDEO -> Classificar por quartis (0 - 1h30 min curto / 1h30-3h medio / 3h+ longo)
df.head()

,CANAL,VIDEO_TITULO,ID_VIDEO,DURACAO,DATA_PUBLICACAO,VISUALIZACOES,CURTIDAS,COMENTARIOS,CATEGORIA_ID,DESCRICAO_CATEGORIA,DT_EXECUCAO_SCRIPT,PODCAST_CONVIDADO,PODCAST_EPISODIO,CURTIDAS_POR_VISUALIZACOES,COMENTARIOS_POR_VISUALIZACOES,DATA_PUBLICACAO_BR,MES_ANO_PUBLICACAO,ANO_PUBLICACAO,MES_PUBLICACAO,HORARIO_PUBLICACAO,DIA_DA_SEMANA,MOMENTO_DIA,DIAS_DESDE_PUBLICACAO
0,Flow 2.0,REINALDO AZEVEDO - FLOW #455,a5M5Je7yKqE,11272.0,2025-06-07T04:00:25Z,234101,20631,1560,22,People & Blogs,2025-06-10 11:10:33,REINALDO AZEVEDO,455,0.088129,0.006664,2025-06-07 01:00:25,06/2025,2025,06,01:00,sábado,MADRUGADA,10
1,Flow 2.0,PVC - FLOW #453,8alvXuLKQzA,9036.0,2025-06-04T00:12:44Z,191972,10072,359,22,People & Blogs,2025-06-10 11:10:33,PVC,453,0.052466,0.001870,2025-06-03 21:12:44,06/2025,2025,06,21:12,terça-feira,NOITE,13
2,Flow 2.0,LORD VINHETEIRO - FLOW #452,7tEuttOcddQ,9618.0,2025-05-30T01:04:27Z,159953,9388,486,22,People & Blogs,2025-06-10 11:10:33,LORD VINHETEIRO,452,0.058692,0.003038,2025-05-29 22:04:27,05/2025,2025,05,22:04,quinta-feira,NOITE,18
3,Flow 2.0,JULIO BALESTRIN E RICHARD RASMUSSEN - FLOW #450,MUogXEAt9_U,8734.0,2025-05-27T01:53:46Z,523483,33905,543,22,People & Blogs,2025-06-10 11:10:33,JULIO BALESTRIN E RICHARD RASMUSSEN,450,0.064768,0.001037,2025-05-26 22:53:46,05/2025,2025,05,22:53,segunda-feira,NOITE,21
4,Flow 2.0,DANIEL LOPEZ - FLOW #449,pThrdd0jyEA,10289.0,2025-05-23T00:45:11Z,599470,35956,1479,22,People & Blogs,2025-06-10 11:10:33,DANIEL LOPEZ,449,0.059980,0.002467,2025-05-22 21:45:11,05/2025,2025,05,21:45,quinta-feira,NOITE,25


In [17]:
df_faltantes = identificar_episodios_faltantes(df, 'CANAL', 'PODCAST_EPISODIO')
df_faltantes

,CANAL,EPISODIO_FALTANTE
0,Flow 1.0,45
1,Flow 1.0,55
2,Flow 1.0,75
3,Flow 1.0,81
4,Flow 1.0,139
5,Flow 1.0,164
6,Flow 1.0,168
7,Flow 1.0,183
8,Flow 1.0,184
9,Flow 1.0,199


## **SALVAR TABELA**

In [ ]:
df.to_excel("../../dados/tabelas/02_EPS_COMPLETOS_CLEAN.xlsx", index=False)
df.head()

,CANAL,VIDEO_TITULO,ID_VIDEO,DURACAO,DATA_PUBLICACAO,VISUALIZACOES,CURTIDAS,COMENTARIOS,CATEGORIA_ID,DESCRICAO_CATEGORIA,DT_EXECUCAO_SCRIPT,PODCAST_CONVIDADO
0,Flow 2.0,REINALDO AZEVEDO - FLOW #455,a5M5Je7yKqE,11272.0,2025-06-07T04:00:25Z,234101,20631,1560,22,People & Blogs,2025-06-10 11:10:33,REINALDO AZEVEDO
1,Flow 2.0,PVC - FLOW #453,8alvXuLKQzA,9036.0,2025-06-04T00:12:44Z,191972,10072,359,22,People & Blogs,2025-06-10 11:10:33,PVC
2,Flow 2.0,LORD VINHETEIRO - FLOW #452,7tEuttOcddQ,9618.0,2025-05-30T01:04:27Z,159953,9388,486,22,People & Blogs,2025-06-10 11:10:33,LORD VINHETEIRO
3,Flow 2.0,JULIO BALESTRIN E RICHARD RASMUSSEN - FLOW #450,MUogXEAt9_U,8734.0,2025-05-27T01:53:46Z,523483,33905,543,22,People & Blogs,2025-06-10 11:10:33,JULIO BALESTRIN E RICHARD RASMUSSEN
4,Flow 2.0,DANIEL LOPEZ - FLOW #449,pThrdd0jyEA,10289.0,2025-05-23T00:45:11Z,599470,35956,1479,22,People & Blogs,2025-06-10 11:10:33,DANIEL LOPEZ
